In [ ]:
# Import modules
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Enrichissement des données
from sklearn.impute import SimpleImputer
# Random Forest
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

%matplotlib inline

### Preprocessing

In [ ]:
data = pd.read_csv("data/adult.csv", na_values='?')

In [ ]:
# Traitements qui sont actés
def remove_columns(data):
    """Remove useless columns"""
    data.drop('education', axis=1, inplace=True)
    data.drop('fnlwgt', axis=1, inplace=True)
    return

def complete_except_occupation(data):
    """ Fill missing data in some columns with a SimpleImputer"""
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    data['workclass'] = imp.fit_transform(data[['workclass']])
    data['native.country'] = imp.fit_transform(data[['native.country']])
    return

def transform(data):
    """Transform multi categorical data into binary data (to properly apply prediction models)"""
    data['native.country'] = [1 if x == 'United-States' else 0 for x in data['native.country']]
    data['workclass'] = [1 if x == 'Private' else 0 for x in data['workclass']]
    data['race'] = [1 if x == 'White' else 0 for x in data['race']]
    data['sex'] = [1 if x == 'Male' else 0 for x in data['sex']]
    data['income'] = [0 if x == '<=50K' else 1 for x in data['income']]

In [1]:
# à relire et tester si tout est bien là
def complete_occupation(data):
    """Fill missing data in 'occupation' column with a KNN prediction """
    df = data.copy()
    df.drop('income', axis=1, inplace=True)
    # Séparation des sets : train et prédiction
    train = df.dropna()
    prediction = pd.get_dummies(df[df['occupation'].isnull()].drop('occupation', axis=1))
    # Prédiction avec un modèle KNN
    X = pd.get_dummies(train.drop('occupation', axis=1))
    Y = train['occupation']
    clf_knn = KNeighborsClassifier(n_neighbors=20)
    clf_knn.fit(X,Y)
    result_knn = clf_knn.predict(prediction)
    # Inscrire les résultats de la prédiction dans le df 'prediction'
    prediction['occupation']=[x for x in result_knn]
    # Compléter les données manquantes du df initial ('data') grâce aux données prédites ('prediction')
    for index in prediction.index:
        data.loc[index,'occupation'] = prediction.loc[index,'occupation']
    return

# + ajouter calcul de score de ce KNN (qui sera modifié si on modifie les prétraitements au dessus)

à regarder pour le preprocessing (dans le modèle et dans le KNN occupation)
- workclass : private - public - independent
- marital status et relationship : voir comment on regroupe, lequel on supprime (a priori marié - séparé - never married)
- capital loss ? 


à regarder pour modifier le modèle : 
- max features (auto, à modifier ?)

### Modèle de prédiction Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_features='auto') #auto = racine de p

Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators = [i for i in range(20,301,20)]
parameters = {'n_estimators': n_estimators}
clf_gs_rf = GridSearchCV(clf_rf, parameters, cv=5, scoring='accuracy')
clf_gs_rf.fit(X, Y)

In [ ]:
clf_gs_rf.cv_results_

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=120, max_features='auto')
scores = cross_val_score(clf_rf, X, Y, cv=5, scoring='accuracy')
np.mean(scores), np.std(scores)

In [ ]:
scores = cross_val_score(clf_rf, X, Y, cv=5, scoring='f1')
np.mean(scores), np.std(scores)

In [2]:
# permet de calculer l'importance de chaque feature : à regarder de près
clf_rf.fit(X,Y)
clf_rf.feature_importances_

NameError: name 'clf_rf' is not defined

In [ ]:
X.head()